# 1 Getting Ready

## 1.1 Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from numpy import arange
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from fpdf import FPDF 
from sklearn.pipeline import Pipeline
import pickle
import datetime
from datetime import datetime as dt
from datetime import timedelta
from keras.models import model_from_json

## 1.2 Setting up Environment

In [2]:
%matplotlib inline
plt.style.use('fivethirtyeight')
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# 2 Load Preprocessed Data for Model Building

## 2.1 Load from pickle

In [3]:
df_churn = pd.read_pickle('./data/training/churn.pickle')

## 2.2 Have a Look into the Training Data

In [4]:
df_churn.head()

,TENURE,MONTHLY_CHARGES,CHURN_STATUS
0,1,29.85,0
1,34,56.95,0
2,2,53.85,1
3,45,42.30,0
4,2,70.70,1


## 2.3 Make a copy of Training Data to work on

In [5]:
df = df_churn.copy()

# 3 Learn more about the Dataset - Meta Info

In [6]:
df.shape

(7043, 3)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 3 columns):
TENURE             7043 non-null int64
MONTHLY_CHARGES    7043 non-null float64
CHURN_STATUS       7043 non-null int64
dtypes: float64(1), int64(2)
memory usage: 165.1 KB


In [8]:
df.columns

Index(['TENURE', 'MONTHLY_CHARGES', 'CHURN_STATUS'], dtype='object')

In [9]:
df.describe()

,TENURE,MONTHLY_CHARGES,CHURN_STATUS
count,7043.000000,7043.000000,7043.000000
mean,32.371149,64.761692,0.265370
std,24.559481,30.090047,0.441561
min,0.000000,18.250000,0.000000
25%,9.000000,35.500000,0.000000
50%,29.000000,70.350000,0.000000
75%,55.000000,89.850000,1.000000
max,72.000000,118.750000,1.000000


# 4 Prepare X (Set of Independent Variables or Features) and y (Dependent or Target Variable)

In [10]:
X = df.drop('CHURN_STATUS', axis=1)
y = df['CHURN_STATUS'].values

# 5 Splitting Data into Train, Validation and Test Set

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.25, random_state=0)

# 6 Oversampling for balancing data

NOT NOW!

# 7 Logistic  Regression

## 7.1 Hyper Params Estimation: Logistic Regression

In [13]:
print("Starting ------ Logistic Regression")

pipe = Pipeline(steps=[
    ('logistic', LogisticRegression())
])

param_grid ={
    'logistic__penalty':('l1', 'l2', 'elasticnet', 'none'),
    'logistic__solver':('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'),
    'logistic__max_iter':[50,100,500]
}

model=GridSearchCV(estimator=pipe,
                         param_grid=param_grid,
                         scoring='roc_auc', 
                         n_jobs=-1,
                         pre_dispatch='2*n_jobs', 
                         cv=5, 
                         verbose=1,
                         return_train_score=False)

model.fit(X_train,y_train)

pkl_filename = "./models/logistic_regression.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

X_val_np = X_val.to_numpy()
predicted = model.predict(X_val_np)
proba = model.predict_proba(X_val_np)

accuracy=accuracy_score(y_val, predicted)

CM = confusion_matrix(y_val, predicted)
(TN,FN,TP,FP) = (CM[0][0],CM[1][0],CM[1][1],CM[0][1])
FPR = FP/(FP+TN)



print("Best Params-")
print(model.best_params_)

print("\n")
print("Algorithm: ", 'Logistic Regression')
print("Accuracy: ",accuracy)
# print("Recall: ", recall)
# print("F1-support: ", f1)
# print("FPR: ", FPR)
# print("Runtime: ", delta)

print("Best Params-")
print(model.best_params_)

Starting ------ Logistic Regression
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best Params-
{'logistic__max_iter': 500, 'logistic__penalty': 'l1', 'logistic__solver': 'saga'}


Algorithm:  Logistic Regression
Accuracy:  0.7792760823278921
Best Params-
{'logistic__max_iter': 500, 'logistic__penalty': 'l1', 'logistic__solver': 'saga'}


C:\Users\Hridoy Alam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.81411277        nan 0.80158115 0.81406292
 0.81406292 0.81405719 0.80795726 0.80155976        nan        nan
        nan        nan        nan 0.81406292 0.81406292        nan
 0.80793298 0.80157832        nan        nan 0.81411277        nan
 0.80791733 0.81406292 0.81406292 0.81405719 0.81264238 0.80790023
        nan        nan        nan        nan        nan 0.81406292
 0.81406292        nan 0.81264665 0.80792727        nan        nan
 0.81410992        nan 0.8141156  0.81406292 0.81406292 0.81405719
 0.81402871 0.81407428        nan        nan        nan        nan
        nan 0.81406292 0.81406292        nan 0.81403297 0.81410704]
  category=UserWarning


## 7.2 Build Baseline Model: Logistic Regression

DO IT YOURSELF

# Thank You